In [1]:
import ee
import geemap
import time
import requests
import json
import os
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd
import topojson as tp
ee.Authenticate()
#geemap.ee_search()
#ee.Initialize()
Map = geemap.Map()


Successfully saved authorization token.


In [19]:
def s2_func(i):
  print(f'inicia salida para temporada {i}')
  t1=int('{}'.format(i))
  prefire_start = str(t1-1)+'-09-01'
  prefire_end = str(t1-1)+'-11-30'

  # Now set the same parameters for AFTER the fire.
  postfire_start = str(t1-1)+'-12-01'
  #postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
  postfire_end = str(t1)+'-07-31'
  area =  ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')
  dem = ee.Image('USGS/SRTMGL1_003')
  dem_mask=dem.gt(2000)
  fires_firms = ee.ImageCollection('FIRMS')\
          .select('T21')\
          .filterDate(prefire_start,postfire_end)\
          .filterBounds(area)

  fires_firmsmx = fires_firms.max().clip(area).subtract(273.15)
  fires_firmsrc = fires_firmsmx.gte(50)
  fires_firmsvc= fires_firmsrc.reduceToVectors(**{
    'geometry':area,
    'crs': 'EPSG:32718',
    'scale':500,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
  }
    )

  fires_firmsvc=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))\
      .filterMetadata('label', 'greater_than', 0)\
      .filterMetadata('count', 'greater_than', 10)
  fires_firmsvc = fires_firmsvc.map(lambda f: f.buffer(500))
  fires_firmsvc = fires_firmsvc.union()
  def addNBR(image):
      nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
      image = image.addBands(nbr)
      return image
  prefireImCol = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterDate(prefire_start, prefire_end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                    .filterBounds(area) \
                    .map(addNBR)
                    
  postfireImCol = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterDate(postfire_start, postfire_end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                    .filterBounds(area) \
                    .map(addNBR)

  prefire = prefireImCol.mosaic().clip(fires_firmsvc)
  postfire = postfireImCol.mosaic().clip(fires_firmsvc)

  preNBR = prefire.select('nbr')
  postNBR = postfire.select('nbr')
  dNBR_withoutsmooth = preNBR.subtract(postNBR)

  dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
      reducer=ee.Reducer.mean(),
      kernel=ee.Kernel.gaussian(radius=20),
  )

  #dNBR_unscaled=dNBR_unscaled.clip(r20)
  dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
  #dNBR_Seguro = dNBR_unscaled.gte(0.27)

  def export_drive(z):
    zona='{z}'.format(z=z)
    area = ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')\
        .filter(ee.Filter.equals("PaisajeID", int(zona)))
    dNBR_vector= dNBR_reclass.reduceToVectors(**{
      'geometry':area,
      'crs': 'EPSG:32718',
      'scale':30,
      'geometryType':'polygon',
      'eightConnected': False,
      'bestEffort':False,
      'maxPixels':9e10,
    }
      )
    dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))
    geemap.ee_export_vector_to_drive(
        dNBR_vector, description=postfire_end+"_dNBR_" + zona, fileFormat='GeoJSON', folder="dNBR"
  )
  print('exportando las zonas')
  for i in range(1,24):
     export_drive(i)


In [4]:
def l8_func(i):
    t1=int('{}'.format(i))
    # filter Landsat 8 SR imagery 
    prefire_start = str(t1-1)+'-09-01'
    prefire_end = str(t1-1)+'-11-30'

    # Now set the same parameters for AFTER the fire.
    postfire_start = str(t1-1)+'-12-01'
    #postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
    postfire_end = str(t1)+'-07-31'

    def addNBR(image):
        nbr = image.normalizedDifference(['B5', 'B7']).rename("nbr")
        image = image.addBands(nbr)
        return image
    area =  ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')
    dem = ee.Image('USGS/SRTMGL1_003')
    dem_mask=dem.gt(2000)
    fires_firms = ee.ImageCollection('FIRMS')\
            .select('T21')\
            .filterDate(prefire_start,postfire_end)\
            .filterBounds(area)

    fires_firmsmx = fires_firms.max().clip(area).subtract(273.15)
    fires_firmsrc = fires_firmsmx.gte(50)
    fires_firmsvc= fires_firmsrc.reduceToVectors(**{
    'geometry':area,
    'crs': 'EPSG:32718',
    'scale':500,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
    }
    )

    fires_firmsvc=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))\
        .filterMetadata('label', 'greater_than', 0)\
        .filterMetadata('count', 'greater_than', 10)
    fires_firmsvc = fires_firmsvc.map(lambda f: f.buffer(500))
    fires_firmsvc = fires_firmsvc.union()

    prefireImCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                    .filterDate(prefire_start, prefire_end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                    .filterBounds(area) \
                    .map(addNBR)
                    
    postfireImCol = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                    .filterDate(postfire_start, postfire_end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                    .filterBounds(area) \
                    .map(addNBR)

    prefire = prefireImCol.mosaic().clip(fires_firmsvc)
    postfire = postfireImCol.mosaic().clip(fires_firmsvc)
    preNBR = prefire.select('nbr')
    postNBR = postfire.select('nbr')
    dNBR_withoutsmooth = preNBR.subtract(postNBR)

    dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
        reducer=ee.Reducer.mean(),
        kernel=ee.Kernel.gaussian(radius=20),
    )

    dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))

    dNBR_vector= dNBR_reclass.reduceToVectors(**{
    'geometry':area,
    'crs': 'EPSG:32718',
    'scale':30,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
    }
    )
    dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))

    geemap.ee_export_vector_to_drive(
        dNDBR_export, description=str(t1)+"_dNBR_500m", fileFormat='GeoJSON', folder="dNBR"
    )

In [5]:
def l5_func(i):
    t1=int('{}'.format(i))
    # filter Landsat 8 SR imagery 
    prefire_start = str(t1-1)+'-09-01'
    prefire_end = str(t1-1)+'-11-30'

    # Now set the same parameters for AFTER the fire.
    postfire_start = str(t1-1)+'-12-01'
    #postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
    postfire_end = str(t1)+'-07-31'

    def addNBR(image):
        nbr = image.normalizedDifference(['B5', 'B7']).rename("nbr")
        image = image.addBands(nbr)
        return image
    area =  ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')
    dem = ee.Image('USGS/SRTMGL1_003')
    dem_mask=dem.gt(2000)
    fires_firms = ee.ImageCollection('FIRMS')\
            .select('T21')\
            .filterDate(prefire_start,postfire_end)\
            .filterBounds(area)

    fires_firmsmx = fires_firms.max().clip(area).subtract(273.15)
    fires_firmsrc = fires_firmsmx.gte(50)
    fires_firmsvc= fires_firmsrc.reduceToVectors(**{
    'geometry':area,
    'crs': 'EPSG:32718',
    'scale':500,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
    }
    )

    fires_firmsvc=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))\
        .filterMetadata('label', 'greater_than', 0)\
        .filterMetadata('count', 'greater_than', 10)
    fires_firmsvc = fires_firmsvc.map(lambda f: f.buffer(500))
    fires_firmsvc = fires_firmsvc.union()

    prefireImCol = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                    .filterDate(prefire_start, prefire_end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                    .filterBounds(area) \
                    .map(addNBR)
                    
    postfireImCol = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                    .filterDate(postfire_start, postfire_end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                    .filterBounds(area) \
                    .map(addNBR)

    prefire = prefireImCol.mosaic().clip(fires_firmsvc)
    postfire = postfireImCol.mosaic().clip(fires_firmsvc)
    preNBR = prefire.select('nbr')
    postNBR = postfire.select('nbr')
    dNBR_withoutsmooth = preNBR.subtract(postNBR)

    dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
        reducer=ee.Reducer.mean(),
        kernel=ee.Kernel.gaussian(radius=20),
    )

    dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))

    dNBR_vector= dNBR_reclass.reduceToVectors(**{
    'geometry':area,
    'crs': 'EPSG:32718',
    'scale':30,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
    }
    )
    dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))

    geemap.ee_export_vector_to_drive(
        dNDBR_export, description=str(t1)+"_dNBR_500m", fileFormat='GeoJSON', folder="dNBR"
    )

In [20]:
s2_func(2024)

inicia salida para temporada 2024
exportando las zonas
Exporting 2024-07-31_dNBR_1... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_2... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_3... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_4... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_5... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_6... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_7... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_8... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_9... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2024-07-31_dNBR_10... Please check the Task Manager from the JavaScript Code Editor.
Expo

In [10]:
temporada=list(range(2001,2024))

In [15]:
for i in temporada:
    if i>2016:
        s2_func(i)
    elif i>2012:
        l8_func(i)
    else:
        l5_func(i)

Exporting 2001_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2002_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2003_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2004_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2005_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2006_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2007_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2008_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2009_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2010_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.
Exporting 2011_dNBR_500m... Please check the Task Manager from the JavaScript Code Editor.